# Save profiles from mulitple files in a netcdf file



## All the imports

In [6]:
import glob

#To deal with data
import xarray as xr

#For basic calculus
import numpy as np
import numpy.ma as ma



## Paths to data

In [7]:
#the path where all data are stored
path_occiput_obs='/mnt/alberta/equipes/IGE/meom/MODEL_SET/ORCA025.L75/ORCA025.L75-OCCITENS/OBS'


## Functions

In [8]:
def selec_one_argo_one_year_all_runs(station,year):
    #first run first
    path=path_occiput_obs+'/OBS-enact_y'+str(year)
    file1=path+'/ORCA025.L75-OCCITENS.001_y'+str(year)+'_enact_fdbk.nc'
    ds1=xr.open_dataset(file1,decode_times=False)
    stations=ds1.STATION_IDENTIFIER
    ind=np.where(stations==station)
    ds1_argo_selec=ds1.sel(N_OBS=ind[0]) 

    POTM_Hx_concat=ds1_argo_selec.POTM_Hx
    PSAL_Hx_concat=ds1_argo_selec.PSAL_Hx
    
    #loop on all other years and concatenate
    for k in np.arange(1,50):
        kk="{:03d}".format(k)
        file=path+'/ORCA025.L75-OCCITENS.'+str(kk)+'_y'+str(year)+'_enact_fdbk.nc'
        ds=xr.open_dataset(file,decode_times=False)
        ds_argo_selec=ds.sel(N_OBS=ind[0])
        POTM_Hx=ds_argo_selec.POTM_Hx
        PSAL_Hx=ds_argo_selec.PSAL_Hx
        POTM_Hx_concat=xr.concat([POTM_Hx_concat, POTM_Hx], "ensemble")
        PSAL_Hx_concat=xr.concat([PSAL_Hx_concat, PSAL_Hx], "ensemble")
        
    #replace model data in the first file by the concatenated 50 runs with dimension ensemble
    ds_argo_selec=ds1_argo_selec.drop_vars({'POTM_Hx','PSAL_Hx'})
    ds_argo_selec['POTM_Hx']=POTM_Hx_concat
    ds_argo_selec['PSAL_Hx']=PSAL_Hx_concat
    return ds_argo_selec
    
    
def selec_one_argo_years_all_runs(station,year1,year2):
    #first year first
    ds_concat=selec_one_argo_one_year_all_runs(station,year1)
    #loop on the other years
    for year in np.arange(year1+1,year2+1):
        ds=selec_one_argo_one_year_all_runs(station,year)
        ds_concat=xr.concat([ds_concat,ds],dim='N_OBS')
        
    #write on disk
    stat=str(station.values)
    str_stat=stat[2:7]
    ds_concat.to_netcdf('/mnt/alberta/equipes/IGE/meom/workdir/alberta/ARGO/selec_profiles/ARGO'+str(str_stat)+'_y'+str(year1)+'-'+str(year2)+'_r1-50.nc')


## Test avec un profil

de 2010 à 2012 150 niveaux verticaux et de 2013 à 2015 400 niveaux

### Select one argo float in one year file

In [9]:
#test for one year
path2010=path_occiput_obs+'/OBS-enact_y2010'
one_file2010=path2010+'/ORCA025.L75-OCCITENS.001_y2010_enact_fdbk.nc'
ds_one2010=xr.open_dataset(one_file2010,decode_times=False)
stations=ds_one2010.STATION_IDENTIFIER
stat44032=stations[100]

### Process the concatenation of 3 years in one file

In [10]:
%%time
selec_one_argo_years_all_runs(stat44032,2010,2012)

CPU times: user 1min 52s, sys: 29 s, total: 2min 21s
Wall time: 9min 42s


In [ ]:
%%time
selec_one_argo_years_all_runs(stat44032,2013,2015)